# BreastDCEDL Explore Dicom data


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/naomifridman/BreastDCEDL/blob/main/ISPY2/explore_spy2_dicom.ipynb)


#### Author: Naomi Fridman
#### Date: 2025-08-12
> BreastDCEDL/BreastDCEDL/explore_spy2_dicom.ipynb

### Avalable data for deno
* Samples from git: ISPY2/data_samples


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import sys
from pathlib import Path

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
# cb - cotrast_brightness
def show_n_images(imgs, cmap='gray', titles = None, enlarge = 4, mtitle=None,
                  cut = 0, axis_off = True, fontsize=15, cb = 0):

    plt.set_cmap(cmap);

    n = len(imgs);
    gs1 = gridspec.GridSpec(1, n);

    fig1 = plt.figure(figsize=(4*len(imgs),8));
    for i in range(n):

        ax1 = fig1.add_subplot(gs1[i]);
        if (cb):
            if len(np.unique(imgs[i])<=5):
                 img = imgs[i]
            else:

                img = cont_br(imgs[i])
        else:
            img = imgs[i]
        if cut:
            ax1.imshow(img[50:290, 75:450] , interpolation='none', origin='lower');
        else:

            ax1.imshow(img, interpolation='none');
        if (titles is not None):
            ax1.set_title(titles[i], fontsize=fontsize);  #, fontweight="bold");
        if (axis_off):
            plt.axis('off')
    if mtitle:
        plt.title(mtitle)
    plt.tight_layout()
    plt.show();

In [ ]:
# Add this at the beginning of your notebook

# Check if running in Google Colab

if 'google.colab' in str(get_ipython()):
    print("Running in Google Colab")
    # Clone the repository
    !git clone https://github.com/naomifridman/BreastDCEDL.git
    !pip install pydicom

    # Change to the repository directory
    os.chdir('/content/BreastDCEDL')


In [ ]:
!ls

In [ ]:
!ls ISPY2

In [ ]:
!ls ISPY2/TCIA_metadata

In [ ]:

from PIL import Image
import requests
import argparse
import datetime
import json
import numpy as np
import os
import time
from pathlib import Path


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
import warnings
warnings.filterwarnings('ignore', '.*do not.*', )
warnings.warn('Do not show this message')

import os
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
from glob import glob
#from skimage import io
from sklearn.utils import shuffle

#from nipype.interfaces.ants import N4BiasFieldCorrection
import sys
import os
import ast

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Dicom utilies

In [ ]:
import pydicom

# return list of numpy array of voxels
def read_dcm_series(fname, bpath=''):
    # Copyright (c) 2025 Naomi Fridman - Licensed under CC BY 4.0
    # (https://creativecommons.org/licenses/by/4.0/)
    fpathf = join_path(bpath,fname)

    ll = os.listdir(fpathf)
    pe=[]
    for p in ll:
            d = pydicom.dcmread(join_path(fpathf,p))
            pe.append(d.pixel_array)
    return pe

# returns list of dcm objectys
def read_dcm_series_dicom(fname, bpath=''):
    # Copyright (c) 2025 Naomi Fridman - Licensed under CC BY 4.0
    # (https://creativecommons.org/licenses/by/4.0/)
    fpathf = join_path(bpath,fname)

    ll = os.listdir(fpathf)
    pe=[]
    for p in ll:
            d = pydicom.dcmread(join_path(fpathf,p))
            pe.append(d)
    return pe

def get_dicom_val(d, tag=(0x0020, 0x0105)):
    # Copyright (c) 2025 Naomi Fridman - Licensed under CC BY 4.0
    # (https://creativecommons.org/licenses/by/4.0/)
    val = d.get(tag).value if tag in d else None
    return val

def get_int_dicom_val(d, tag=(0x0020, 0x0105)):
    # Copyright (c) 2025 Naomi Fridman - Licensed under CC BY 4.0
    # (https://creativecommons.org/licenses/by/4.0/)
    val = int(float(d.get(tag).value)) if tag in d else None
    return val

In [ ]:
def extract_image_positions(dicom):
    # Copyright (c) 2025 Naomi Fridman - Licensed under CC BY 4.0
    # (https://creativecommons.org/licenses/by/4.0/)

    """
    Extracts Image Position (Patient) values from a multi-frame DICOM file
    and returns them as a pandas DataFrame.

    Parameters:
    dicom: DICOM file.

    Returns:
    pd.DataFrame: DataFrame containing frame number and (X, Y, Z) positions.
    """
    # Load the DICOM file
    dicom_file = dicom

    # Extract all image positions
    image_positions = []

    if "PerFrameFunctionalGroupsSequence" in dicom_file:
        for frame_idx, frame in enumerate(dicom_file.PerFrameFunctionalGroupsSequence):
            if "PlanePositionSequence" in frame:
                position = list(frame.PlanePositionSequence[0].ImagePositionPatient)  # Convert MultiValue to list
                image_positions.append([frame_idx] + position)

    # Create a DataFrame
    df_positions = pd.DataFrame(image_positions, columns=["Frame", "X", "Y", "Z"])

    return df_positions

# Load metadata

In [ ]:
df = pd.read_csv(os.path.join('BreastDCEDL_metadata.csv'))
df.head()

# Load example and visualize

In [ ]:
subject_id = 'ISPY2-550421'

df[df.pid==subject_id].T

# Read DICOM
### Load TCIA metadata

In [ ]:
df_meta = pd.read_csv(os.path.join('ISPY2','TCIA_metadata','metadata.csv'))
df_meta.head()

In [ ]:
def get_visit(x):
    if x[-2:] == 'T0':
        return 'T0'
    elif x[-2:] == 'T1':
        return 'T1'
    elif x[-2:] == 'T2':
        return 'T2'
    elif x[-2:] == 'T3':
        return 'T3'

## Filter Pre Treatment visit

In [ ]:
df_meta['Study Description'] = df_meta['Study Description'].apply(lambda x: get_visit(x))

df_meta['Study Description'].value_counts()

df_meta=df_meta[df_meta['Study Description']=='T0']

# Analiz few examples

In [ ]:
!ls ISPY2/data_samples/dicom_tcia/ISPY2

In [ ]:
p='ISPY2-208265'
p='ISPY2-550421'
tmp=df_meta[df_meta['Subject ID']==p]
tmp[tmp['Series Description']=='ISPY2 VOLSER uni-lateral cropped original DCE']


In [ ]:
dcm_path = os.path.join("ISPY2","data_samples","dicom_tcia")

In [ ]:
import os
from pathlib import Path

# Extract and clean the raw path
raw_fname = tmp[tmp['Series Description'] == 'ISPY2 VOLSER uni-lateral cropped original DCE']['File Location'].values[0]

# Convert to a proper path object (cross-platform safe)
fname = Path(raw_fname.replace("\\", os.sep).lstrip(".\\/"))

# Build full path
full_path = Path(dcm_path) / fname

# Check and list
if full_path.exists():
    print(list(full_path.iterdir())[:2])
else:
    print("Path does not exist:", full_path)

In [ ]:
!ls ISPY2/data_samples/dicom_tcia/ISPY2/ISPY2-550421/

In [ ]:
!ls ISPY2/data_samples/dicom_tcia/ISPY2/ISPY2-550421/1.3.6.1.4.1.14519.5.2.1.92610516567030038611570123081828666823

In [ ]:
import os
import pydicom
from pathlib import Path

# Normalize fname and build full directory path
clean_fname = str(fname).replace("\\", "/")
dicom_dir = Path(dcm_path) / Path(clean_fname)

# Get the first file in the directory
first_file = sorted(os.listdir(dicom_dir))[0]  # sorted ensures consistent order

# Build full path to the DICOM file
dicom_path = dicom_dir / first_file

# Read the DICOM file
d = pydicom.dcmread(dicom_path)

In [ ]:
d

```
(0019,1017) [SliceResolution]                   DS: '0.614583'
 (0028,0030) Pixel Spacing                       DS: [0.7291666865, 0.7291666865]   
 (0018,0088) Spacing Between Slices              DS: '1'
```

In [ ]:

tmp=df_meta[df_meta['Subject ID']==p]
tmp[tmp['Series Description']=='ISPY2 VOLSER uni-lateral cropped original DCE']


In [ ]:
df[df.pid==p].T

```
(0019,1017) [SliceResolution]                   DS: '0.614583'
 (0028,0030) Pixel Spacing                       DS: [0.7291666865, 0.7291666865]   

 (0028,0030) Pixel Spacing                       DS: [0.9375, 0.9375]
 (0018,0050) Slice Thickness                     DS: '1'
 (0018,0088) Spacing Between Slices              DS: '1'
```

# Analize few example cases

pds = 'ACRIN-6698-378885'
pdg = 'ISPY2-211408'
pdph = 'ISPY2-550421'

### Read Dynamic Contrast MRI for ISPY2-550421

## Dynamic Contrast MRI Scan Details for ISPY2-550421

- **Total Slices:** 1120  
- **Number of MRI Scans:** 7  
- **Slices per MRI Scan:** 1120 / 7 = **160 slices per scan**  

#### **Acquisition Timing**
- The **first scan** is taken **before contrast injection**.  
- The **remaining 6 scans** are acquired **after injection**, with a time interval of approximately **1.5 minutes** between each.  

#### **Processing Steps**
1. **Group slices by acquisition time** (i.e., pre-contrast and post-contrast phases).  
2. **Order slices spatially** based on their anatomical position to ensure correct reconstruction.  


## 1. **Group slices by acquisition time** (i.e., pre-contrast and post-contrast phases).  

In [ ]:
d

## Dicom Temporal information
### slice 1-0001.dcm
```
(0020,0013) Instance Number                     IS: '1'
(0020,0032) Image Position (Patient)            DS: [-20.2284866531, -133.1898132546, 62.1355361938]
(0020,0052) Frame of Reference UID              UI: 1.3.6.1.4.1.14519.5.2.1.19848329376685071829001531299299662674
(0020,0060) Laterality                          CS: 'L'
(0020,0100) Temporal Position Identifier        IS: '0'
(0020,0105) Number of Temporal Positions        IS: '6'
(0020,1040) Position Reference Indicator        LO: ''
(0020,1041) Slice Location                      DS: '213'

(0117,1030) Private tag data                    IS: '7'  # Total phases Number
(0117,1031) Private tag data                    DS: '81.832862854'   # Single phase acquisition duration
(0117,1033) Private tag data                    TM: '073158.20'   # Injection time
# phase encoding effective time half way through the acquisition
(0117,1034) Private tag data                    DS: [-73.8535614014, 40.916431427, 122.2864379883, 203.6464233398, 285.006439209, 366.3764343262, 447.7364501953]
```

### **Understanding the Timing of Scans and Contrast Injection in Dynamic Contrast-Enhanced MRI (DCE-MRI)**

In **DCE-MRI**, multiple image acquisitions (phases) are taken over time **before and after** a contrast agent injection. Your DICOM metadata contains important timing information regarding when the scans are taken relative to the contrast injection.

---

## **Temporal Key DICOM Tags**
### **1️⃣ Total Phases Number (0117,1030) - Private Tag**
- **Type:** `IS` (Integer String)
- **Example Value:** `'7'`
- **Meaning:** This represents the **total number of acquisitions (phases)** in the DCE-MRI study, including **one pre-contrast phase** and **multiple post-contrast phases**.

**Interpretation**:  
- `'7'` means **one pre-contrast** and **six post-contrast** phases were acquired.
- These phases are acquired at different time points **before and after** contrast injection.

---

### **2️⃣ Single Phase Acquisition Duration (0117,1031) - Private Tag**
- **Type:** `DS` (Decimal String)
- **Example Value:** `'81.832862854'` (in seconds)
- **Meaning:** Duration of a **single acquisition phase**.

**Interpretation**:  
- Each phase takes **81.83 seconds** to acquire.
- If there are 7 phases, the total scan time will be **approximately** `7 × 81.83 ≈ 573 seconds (9.5 minutes)`.

---

### **3️⃣ Injection Time (0117,1033) - Private Tag**
- **Type:** `TM` (Time)
- **Example Value:** `'073158.20'` (HHMMSS.FF)
- **Meaning:** The **exact scanner clock time** when contrast injection was administered.

**Interpretation**:  
- The contrast injection happened at **07:31:58.20 AM**.
- This time acts as the **reference point** for determining post-contrast acquisitions.

---

### **4️⃣ Effective Acquisition Delay (0117,1034) - Private Tag**
- **Type:** `DS (1-n)` (List of Decimal Strings)
- **Example Value:**
  ```plaintext
  [-73.85, 40.91, 122.28, 203.65, 285.00, 366.37, 447.73]
  ```
- **Meaning:** Time delays (in seconds) relative to **the first post-contrast phase**.

**Interpretation**:  
- **Negative values (`-73.85`)** indicate pre-contrast acquisition (`~74 sec before injection`).
- **Positive values (`40.91`, `122.28`, etc.)** indicate post-contrast acquisitions at these time points after injection.

---

### **5️⃣ Number of Temporal Positions (0020,0105)**
- **Type:** `IS` (Integer String)
- **Example Value:** `'6'`
- **Meaning:** The **number of distinct time points** recorded in this dynamic MRI scan.

**Interpretation**:  
- There are **6 temporal positions**, which correspond to the number of post-contrast scans.

---

## **When Are the Scans Taken**
Based on these values, here’s the approximate timeline of the scan:

| **Phase** | **Time (seconds)** | **Relative to Injection** |
|-----------|------------------|----------------------|
| **Pre-Contrast** | `-73.85` sec | **Before injection** |
| **1st Post-Contrast** | `40.91` sec | **40.91 sec after injection** |
| **2nd Post-Contrast** | `122.28` sec | **122.28 sec after injection** |
| **3rd Post-Contrast** | `203.65` sec | **203.65 sec after injection** |
| **4th Post-Contrast** | `285.00` sec | **285 sec after injection** |
| **5th Post-Contrast** | `366.37` sec | **366.37 sec after injection** |
| **6th Post-Contrast** | `447.73` sec | **447.73 sec after injection** |

---

### **Summary of Timing & Injection Process**
1. **A pre-contrast scan** is taken **~74 seconds before** contrast injection.
2. **Contrast is injected at** `07:31:58.20 AM`.
3. **Post-contrast scans are taken** at different time points after injection (`40.91 sec`, `122.28 sec`, etc.).
4. The **total number of phases** (scans) is **7** (1 pre-contrast + 6 post-contrast).
5. **Each phase acquisition takes** `81.83 sec`.

---

### **Why This Matters in DCE-MRI**
- The **first post-contrast scan** shows **early enhancement** (blood supply to tissues).
- Later phases help differentiate **benign vs. malignant** lesions based on **washout curves**.
- **Temporal Position Identifier (0020,0100)** tells us which phase each image belongs to.


## 2. **Order slices spatially** based on their anatomical position to ensure correct reconstruction.  

### Dicom spatial information
### slice 1-0001.dcm
```
(0020,0013) Instance Number                     IS: '1'
(0020,0032) Image Position (Patient)            DS: [-20.2284866531, -133.1898132546, 62.1355361938]
(0020,0052) Frame of Reference UID              UI: 1.3.6.1.4.1.14519.5.2.1.19848329376685071829001531299299662674
(0020,1041) Slice Location                      DS: '213'
```

### slice 1-0002.dcm
```
(0020,0013) Instance Number                     IS: '2'
(0020,0032) Image Position (Patient)            DS: [-20.2284866531, -133.1898132546, 61.1355361938]
(0020,0052) Frame of Reference UID              UI: 1.3.6.1.4.1.14519.5.2.1.19848329376685071829001531299299662674
(0020,1041) Slice Location                      DS: '214'
```

### slice 1-1116.dcm
```
(0020,0013) Instance Number                     IS: '1116'
(0020,0032) Image Position (Patient)            DS: [-20.2284866531, -133.1898132546, -92.8644638062]
(0020,1041) Slice Location                      DS: '368'

* slicw 1: (0020,0032) Image Position (Patient)  DS: [-20.2284866531, -133.1898132546, 62.1355361938]
* slice 2: (0020,0032) Image Position (Patient)  DS: [-20.2284866531, -133.1898132546, 61.1355361938]
* slice 1116: (0020,0032) Image Position (Patient)  DS: [-20.2284866531, -133.1898132546, -92.8644638062]

### **Positioning Tags in DICOM**
These tags provide information about the **spatial location and orientation** of the image within the 3D coordinate system of the patient.

---

## **1️⃣ (0020,0032) Image Position (Patient)**
- **Type:** `DS (Decimal String)` – 3 values in millimeters `(X, Y, Z)`
- **Example Value:** `[-20.2284866531, -133.1898132546, 62.1355361938]`
- **Meaning:** This gives the **coordinates of the center of the top-left pixel** (first row, first column) **in the DICOM 3D coordinate system**.

**How It Works:**
- The **DICOM coordinate system** follows:
  - **X**: Left (-) to Right (+)
  - **Y**: Posterior (-) to Anterior (+)
  - **Z**: Inferior (-) to Superior (+)

**Interpretation of the Example:**
- **X = -20.23 mm** → The image is **20.23 mm left** of the reference point.
- **Y = -133.19 mm** → The image is **133.19 mm posterior (towards the back)**.
- **Z = 62.14 mm** → The image is **62.14 mm above the reference point**.

## 2️⃣  (0020,1041) Slice Location in DICOM

🔹 **Tag:** `(0020,1041)`  
🔹 **Type:** `DS (Decimal String)`  
🔹 **Example Value:** `'213'`  
🔹 **Meaning:** Represents the **relative position** of the image slice along the **Z-axis (Superior-Inferior direction)**.

### 🔹 How is Slice Location (0020,1041) Used?

#### **🔹 Relative Positioning**
- `Slice Location` is often used for **sorting slices** in **sequential imaging** (e.g., MRI or CT scans).  
- It provides a **reference value** (not absolute) to determine if a slice is positioned **higher or lower** compared to others.

#### **🔹 Comparison with Image Position (0020,0032)**
- **Image Position (Patient) `(0020,0032)`** provides the **absolute position** in **millimeters** within the patient’s coordinate system.  
- **Slice Location `(0020,1041)`** is often a **simplified or derived value** used for sorting slices within a series.

#### **🔹 Usage in Multi-Phase Imaging (DCE-MRI)**
- Since **(0020,0100) Temporal Position Identifier = '0'**, this is likely the **first temporal position** in a **dynamic contrast-enhanced MRI** scan.  
- The **Slice Locatio



In [ ]:
pds = 'ACRIN-6698-378885'
pdg = 'ISPY2-211408'
pdph = 'ISPY2-550421'

# Convert data for: ACRIN-6698-378885

### Read Analysis mask of MRI for ACRIN-6698-378885

In [ ]:
tmp=df_meta[df_meta['Subject ID']==p]
tmp[tmp['Series Description'].str.contains('Analysis Mask')]


In [ ]:
fname = tmp[tmp['Series Description'].str.contains('Analysis Mask')]['File Location'].values[0]
fname

In [ ]:
import os
import pydicom
from pathlib import Path

# Normalize fname and build full directory path
clean_fname = str(fname).replace("\\", "/")
dicom_dir = Path(dcm_path) / Path(clean_fname)

# Get the first file in the directory
first_file = sorted(os.listdir(dicom_dir))[0]  # sorted ensures consistent order

# Build full path to the DICOM file
dicom_path = dicom_dir / first_file

# Read the DICOM file
mask_dcm = pydicom.dcmread(dicom_path)

In [ ]:
mask_dcm

### Mask dicom contains the positions of all the slices
```
 (0020,9111)  Frame Content Sequence  1 item(s) ----
      (0020,9057) In-Stack Position Number            UL: 160
      
      ---------
   (0020,9113)  Plane Position Sequence  1 item(s) ----
      (0020,0032) Image Position (Patient)            DS: [-152.6144414256, -167.5486850431, -78.4195278883]
      ---------
   ---------
 (0020,9111)  Frame Content Sequence  1 item(s) ----
      (0020,9057) In-Stack Position Number            UL: 108
      ---------
   (0020,9113)  Plane Position Sequence  1 item(s) ----
      (0020,0032) Image Position (Patient)            DS: [-152.6144414256, -167.5486850431, -26.4195278883]
```

In [ ]:
mask_dcm.pixel_array.shape

In [ ]:
df_pos = extract_image_positions(mask_dcm)
df_pos.head()

In [ ]:
df_pos.tail()

In [ ]:
df_pos.describe()

In [ ]:
df.columns

In [ ]:
df[df.pid==p]

7 directories with 160 images each, lets read one dicom to see the information

## Dynamic Contrast MRI Scan Details for ACRIN-6698-378885

- **Total Slices:** 1120  
- **Number of MRI Scans:** 7  
- **Slices per MRI Scan:** 1120 / 7 = **160 slices per scan**  

#### **Acquisition Timing**
- The **first scan** is taken **before contrast injection**.  
- The **remaining 6 scans** are acquired **after injection**, with a time interval of approximately **1.5 minutes** between each.  

#### **Processing Steps**
1. **Group slices by acquisition time** (i.e., pre-contrast and post-contrast phases).  
2. **Order slices spatially** based on their anatomical position to ensure correct reconstruction.  


## 1. **Group slices by acquisition time** (i.e., pre-contrast and post-contrast phases).  
#### Pay attention that data temporal and spatial are orgenized for this paitent in a totally different way then the previous paitent. Here we have 7 directories (1 per each aquisition) and we dont have injection times in the slices,so we need to read one slice from each aquisition directory, to understand the temporal order.

In [ ]:


print('1-001.dcm Series Number', get_dicom_val(d,(0x0020, 0x0011)))
print('1-001.dcm Acquisition Number', get_dicom_val(d,(0x0020, 0x0012)))
print('1-001.dcm Instance Number', get_dicom_val(d,(0x0020, 0x0013)))
print('1-001.dcm Number of temporal aquisitions:', get_dicom_val(d,(0x0020, 0x0103)))

print('1-001.dcm in_stack_pos', get_dicom_val(d,(0x0020, 0x9057)))
print('1-001.dcm Acq_tm', get_int_dicom_val(d,(0x0008, 0x0032)))
print('1-001.dcm Cont_tm', get_int_dicom_val(d,(0x0008, 0x0033)))
print('1-001.dcm temporal_id', d.get((0x0020, 0x0100)).value if (0x0020, 0x0100) in d else None)
print('1-001.dcm n_tmp_pos_post', d.get((0x0020, 0x0105)).value if (0x0020, 0x0105) in d else None)
print('1-001.dcm n_tmp_pos_total', d.get((0x0117, 0x1030)).value if (0x0117, 0x1030) in d else None)

## Dicom Temporal information
### slice 1-0001.dcm
```
(0020,0013) Instance Number                     IS: '1'
(0020,0032) Image Position (Patient)            DS: [-152.6144414256, -167.5486850431, 80.5804721117]
(0020,1041) Slice Location                      DS: '80.5804721117'
(0028,0010) Rows                                US: 384
(0028,0011) Columns                             US: 384
(0028,0030) Pixel Spacing                       DS: [0.78125, 0.78125]
(0020,0011) Series Number                       IS: '21'
(0020,0012) Acquisition Number                  IS: '7'

```

### In this case we need to use other set of dicom tags
### **DICOM Tags Related to Injection Time in Contrast-Enhanced Imaging**

DICOM provides several metadata fields related to **contrast injection time** in **Dynamic Contrast-Enhanced MRI (DCE-MRI)** and **CT scans**. These tags help track the timing of contrast administration relative to image acquisition.

#### **🔹 Key Injection Time-Related DICOM Tags:**

| **Tag**         | **Name**                             | **Type** | **Description** |
|---------------|----------------------------------|------------|----------------------------------------------------------------|
| **(0018,1042)** | **Contrast/Bolus Start Time**   | **TM (Time)**  | The time when the contrast agent injection **started**. |
| **(0018,1043)** | **Contrast/Bolus Stop Time**    | **TM (Time)**  | The time when the contrast agent injection **ended**. |
| **(0018,1041)** | **Contrast/Bolus Volume**       | **DS (Decimal String)** | The **volume of contrast injected**, typically in milliliters (mL). |
| **(0018,1044)** | **Contrast Flow Rate**         | **DS (Decimal String)** | The **rate** of contrast injection (mL/sec). |
| **(0018,1045)** | **Contrast Flow Duration**     | **DS (Decimal String)** | The **total duration** of the contrast injection (seconds). |
| **(0018,1046)** | **Contrast/Bolus Ingredient**  | **LO (Long String)** | The name of the **contrast agent** used (e.g., Gadolinium). |
| **(0018,1047)** | **Contrast/Bolus Concentration** | **DS (Decimal String)** | The **concentration** of contrast agent (e.g., mg/mL). |
| **(0018,9312)** | **Contrast/Bolus Agent Administered** | **CS (Code String)** | Indicates whether the contrast was administered (`YES/NO`). |
| **(0018,9333)** | **Contrast Flow Timing**        | **CS (Code String)** | Defines if contrast administration is **bolus, continuous, or phased**. |
| **(0018,0010)** | **Contrast/Bolus Agent**       | **LO (Long String)** | The name of the **contrast agent**, often the same as (0018,1046). |
| **(0018,0012)** | **Contrast/Bolus Route**       | **CS (Code String)** | How the contrast was administered (e.g., **Intravenous (IV)**). |
| **(0018,0014)** | **Contrast/Bolus Total Dose**  | **DS (Decimal String)** | The total **dose of contrast agent** delivered. |
| **(0018,1071)** | **Radiopharmaceutical Start Time** | **TM (Time)** | If a radiopharmaceutical was used, this tag records its start time. |
| **(0020,0103)** | **Number of Temporal Positions** | **IS (Integer String)** | Number of **time points** acquired in DCE imaging. |
| **(0117,1033)** | **Injection Time (Private Tag)** | **TM (Time)** | Specific scanner-dependent injection time metadata. |

---

### **🔹 Usage in DCE-MRI and Contrast-Enhanced Imaging**
1. **Tracking Contrast Administration**
   - **Start & Stop Times** (`0018,1042` & `0018,1043`) help align imaging time points relative to contrast injection.
   - **Flow Rate & Volume** determine how much contrast was injected.

2. **Temporal Mapping in DCE-MRI**
   - The **contrast agent time stamps** allow calculation of the time delay between contrast injection and image acquisition.
   - The **Temporal Position Identifier (0020,0100)** helps sort images acquired at different time points post-injection.

3. **Enhancement Curve Analysis**
   - In DCE-MRI, contrast dynamics over time are used to generate **kinetic curves** that characterize tumor perfusion.

#### In the current example there is no value for numer of aquisitions or Injection times, and this data needs to be calculated from other information.
## We read one slice from each directory to understand the temporal order of the data

### **Understanding the Timing of Scans and Contrast Injection in Dynamic Contrast-Enhanced MRI (DCE-MRI)**

In **DCE-MRI**, multiple image acquisitions (phases) are taken over time **before and after** a contrast agent injection. The  DICOM metadata contains important timing information regarding when the scans are taken relative to the contrast injection. We can see that the dicom taags for this paitewnt, is different then the previous paitent, so we need to reaserch and order them.

---
You're right - let's start over with analyzing the directories from the beginning. The key identifying tags I'll use are:
- Series Number (0020,0011)
- Acquisition Number (0020,0012)
- Acquisition Time (0008,0032)
- Series Description (0008,103E)

Directory by directory analysis:

I should actually order them by Acquisition Number (0020,0012) rather than Series Number, since Acquisition Number directly indicates the temporal sequence of the scan phases in DCE-MRI. Let me correct the order:

Order by Acquisition Number (0020,0012):

1. Dir 4 (Pre-contrast baseline)
   - Acquisition Number: 1
   - Series Number: 7
   - Acquisition Time: 14:08:18

2. Dir 5 (Early post-contrast)
   - Acquisition Number: 2
   - Series Number: 8
   - Acquisition Time: 14:10:18 (+2min)

3. Dir 2 (Post-contrast)
   - Acquisition Number: 3
   - Series Number: 13
   - Acquisition Time: 14:11:49 (+3.5min)

4. Dir 1 (Post-contrast)
   - Acquisition Number: 4
   - Series Number: 15
   - Acquisition Time: 14:13:19 (+5min)

5. Dir 6 (Post-contrast)
   - Acquisition Number: 5
   - Series Number: 17
   - Acquisition Time: 14:14:50 (+6.5min)

6. Dir 3 (Post-contrast)
   - Acquisition Number: 6
   - Series Number: 19
   - Acquisition Time: 14:16:20 (+8min)

7. Dir 0 (Late post-contrast)
   - Acquisition Number: 7
   - Series Number: 21
   - Acquisition Time: 14:17:51 (+9.5min)

You're right - Acquisition Number is the more reliable indicator of temporal sequence in DCE-MRI since it explicitly represents the order of acquisition phases. While Series Numbers also increase sequentially in this case, the Acquisition Number is specifically designed to track the temporal order of the dynamic acquisition sequence.

This is further validated by the Acquisition Times, which show a consistent progression matching the Acquisition Number sequence rather than Series Number. Thank you for the correction!

Chronological order of acquisitions based on Acquisition Time:
1. Directory 4 (14:08:18) - Pre-contrast baseline
2. Directory 5 (14:10:18) - Early post-contrast (+2min)
3. Directory 2 (14:11:49) - Post-contrast (+3.5min)
4. Directory 1 (14:13:19) - Post-contrast (+5min)
5. Directory 6 (14:14:50) - Post-contrast (+6.5min)
6. Directory 3 (14:16:20) - Post-contrast (+8min)
7. Directory 0 (14:17:51) - Post-contrast (+9.5min)

All images are from the same series description "ISPY2: T13DFL AX DYN +C NEW", indicating they are part of the same dynamic contrast-enhanced sequence.

---

### Understanding the  key differences between **ISPY2-550421** and **ACRIN-6698-378885** :

**ISPY2-550421** Case:
- Used specific private DICOM tags for temporal information:
  - Total Phases Number (0117,1030)
  - Single Phase Acquisition Duration (0117,1031)
  - Injection Time (0117,1033)
  - Effective Acquisition Delay (0117,1034)
  - Number of Temporal Positions (0020,0105)
- Had explicit timing information relative to contrast injection
- Contained exact injection time
- Provided precise delay times for each phase relative to injection
- Data organization: Not specified in previous summary

**ACRIN-6698-378885**:
- Uses standard DICOM tags for temporal ordering:
  - Acquisition Number (0020,0012)
  - Series Number (0020,0011)
  - Acquisition Time (0008,0032)
- No explicit injection time tag
- No direct timing information relative to contrast injection
- Data organization:
  - 7 separate directories (0-6)
  - Each directory contains slices from one acquisition phase
  - Directory numbers or names don't match temporal sequence
  - Must use DICOM tags to determine correct temporal order

Key Differences:
1. Private vs Standard Tags:
   - **ISPY2-550421**: Used private tags (0117,xxxx) with explicit injection timing
   - **ACRIN-6698-378885**: Uses standard DICOM tags for temporal sequencing

2. Timing Information:
   - **ISPY2-550421**: Precise timing relative to injection point
   - **ACRIN-6698-378885**: Only absolute acquisition times, no injection reference

3. Phase Identification:
   - **ISPY2-550421**: Direct phase timing via Effective Acquisition Delay
   - **ACRIN-6698-378885**: Must infer phases from Acquisition Number sequence

## 2. **Order slices spatially** based on their anatomical position to ensure correct reconstruction.  
In this case we have 7 directories, each directorey contains one temporal aquisition. So we need to order slices in each directory, in the correct spatial order, so all scans and the tumor segmentation mask, are ordered in the same manner.

### Dicom spatial information
### Directory 0, slice 1-001.dcm
```
(0020,0032) Image Position (Patient)            DS: [-152.6144414256, -167.5486850431, 80.5804721117]
(0019,1015) [SlicePosition_PCS]                 FD: [-152.61444143, -167.54868504, 79.58047211]
```
### Directory 1, slice 1-001.dcm
```
(0020,0032) Image Position (Patient)            DS: [-152.6144414256, -167.5486850431, 80.5804721117]
(0019,1015) [SlicePosition_PCS]                 FD: [-152.61444143, -167.54868504, 79.58047211]
```
### Directory 2, slice 1-001.dcm
```
(0020,0032) Image Position (Patient)            DS: [-152.6144414256, -167.5486850431, 80.5804721117]
(0019,1015) [SlicePosition_PCS]                 FD: [-152.61444143, -167.54868504, 79.58047211]
(0019,100B) [SliceMeasurementDuration]          DS: '90077.5'
```

### Read Analysis mask of MRI for ACRIN-6698-378885

<p style="text-align: left; direction: ltr;">
</p>

# Read analysis mask

### **🔹 Understanding Mask Value Combinations in I-SPY 2**
Your mask values **[0, 1, 2, 17, 32, 33, 34, 49]** likely result from **bitwise combinations** of the fundamental segmentation steps.

In **bit-encoded masks**, each voxel can have a **single value or a combination of multiple steps**, meaning that:
- **Some values are standalone** (e.g., `32` for **Manual VOI**).
- **Other values result from bitwise OR operations** (e.g., `32 + 1 = 33`, meaning both **Manual VOI** and **PE Threshold** were applied).

---

### **🔹 Mapping Your Values to Their Segmentation Steps**
| **Mask Value** | **Bitwise Components** | **Interpretation** |
|--------------|------------------|----------------|
| `0`  | No bits set | Background (included in FTV measurement) |
| `1`  | `1` | **PE Threshold applied** |
| `2`  | `2` | **MNC Filter applied** |
| `17` | `1 + 16` (PE Threshold + Unknown bit) | Likely thresholding + additional process |
| `32` | `32` | **Manual VOI applied** |
| `33` | `32 + 1` | **Manual VOI + PE Threshold** |
| `34` | `32 + 2` | **Manual VOI + MNC Filter** |
| `49` | `32 + 16 + 1` (VOI + Unknown + PE Threshold) | Likely final adjusted mask |

📌 **Key Insight:**  
- **Values `33`, `34`, and `49`** suggest that **Manual VOI was combined with other processing steps**.
- **`17` and `49`** likely involve **extra post-processing steps**.

---


### Mask dicom contains the positions of all the slices
```
 ---------
   (0020,9111)  Frame Content Sequence  1 item(s) ----
      (0020,9057) In-Stack Position Number            UL: 96
      ---------
   (0020,9113)  Plane Position Sequence  1 item(s) ----
      (0020,0032) Image Position (Patient)            DS: [-148.7403493235, -147.209702461, -14.4195278883]
      ---------
   ---------
 (0020,9111)  Frame Content Sequence  1 item(s) ----
      (0020,9157) Dimension Index Values              UL: 98
      ---------
   (0020,9113)  Plane Position Sequence  1 item(s) ----
      (0020,0032) Image Position (Patient)            DS: [-148.7403493235, -147.209702461, -16.4195278883]
      ---------
```

<p style="text-align: left; direction: ltr;">
</p>

# Read analysis mask and convert to binary mask

### **🔹 Understanding Mask Value Combinations in I-SPY 2**
Your mask values **[0, 1, 2, 17, 32, 33, 34, 49]** likely result from **bitwise combinations** of the fundamental segmentation steps.

In **bit-encoded masks**, each voxel can have a **single value or a combination of multiple steps**, meaning that:
- **Some values are standalone** (e.g., `32` for **Manual VOI**).
- **Other values result from bitwise OR operations** (e.g., `32 + 1 = 33`, meaning both **Manual VOI** and **PE Threshold** were applied).

---

### **🔹 Mapping Your Values to Their Segmentation Steps**
| **Mask Value** | **Bitwise Components** | **Interpretation** |
|--------------|------------------|----------------|
| `0`  | No bits set | Background (included in FTV measurement) |
| `1`  | `1` | **PE Threshold applied** |
| `2`  | `2` | **MNC Filter applied** |
| `17` | `1 + 16` (PE Threshold + Unknown bit) | Likely thresholding + additional process |
| `32` | `32` | **Manual VOI applied** |
| `33` | `32 + 1` | **Manual VOI + PE Threshold** |
| `34` | `32 + 2` | **Manual VOI + MNC Filter** |
| `49` | `32 + 16 + 1` (VOI + Unknown + PE Threshold) | Likely final adjusted mask |

📌 **Key Insight:**  
- **Values `33`, `34`, and `49`** suggest that **Manual VOI was combined with other processing steps**.
- **`17` and `49`** likely involve **extra post-processing steps**.

---


# From TCIA  documentation

4 or 5 separate masking steps were used in the segmentation of each DCE acquisition:
Manual VOI	A rectangular volume of interest encompassing the enhancing tumor region was drawn on 3 orthogonal MIP images
Background	Background noise areas and saturated fat regions were eliminated using an intensity threshold applied to the pre-contrast DCE image. The threshold was set to a percentage of the 95th percentile of the intensity histogram from the VOI. The percentage used was set empirically for each imaging site in I-SPY1.
PE Threshold	Normal non-enhancing breast tissue was masked with a percent enhancement (PE) threshold on the early PE map. The threshold used was set empirically for each imaging site in I-SPY1.
Connectivity	A 3D minimum neighbor count (MNC) filter was applied to the mask resulting from the background and PE thresholds. Voxels were masked out which did not have at least the MNC of immediate neighbors in the mask. The MNC used was set empirically for each imaging site in I-SPY1.
OMIT regions	Where deemed necessary by a trained observer, manual regions of interested were drawn to omit from the analysis regions non-tumor enhancing regions that were not eliminated by the other masking steps.

The analysis masks provided are bit-encoded to indicate which masking step(s) were applicable to each voxel in the image. Values used for each of the masking steps are:
1	PE Threshold
2	MNC Filter
8	Background mask
32	Manual VOI
64	OMIT Regions




In [ ]:
idx=[75,80,85]
show_n_images([mask_dcm.pixel_array[i] for i in idx],cmap='jet')

In [ ]:
im=mask_dcm.pixel_array[80][70:270,250:350].copy()
im.shape

In [ ]:
im=mask_dcm.pixel_array[80].copy()
im1 = np.where(im==1,1,0)
im2 = np.where(im==17,1,0)
im3 = np.where((im==17 )| (im==1),1,0)
show_n_images([mask_dcm.pixel_array[80],im1,im2,im3])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the gray level to RGB mapping
gray_to_rgb = {
    0: (0, 0, 0),         # Black
    1: (255, 0, 0),       # Red
    2: (0, 255, 0),       # Green
    17: (0, 0, 255),      # Blue
    32: (255, 255, 0),    # Yellow
    33: (255, 0, 255),    # Magenta
    34: (0, 255, 255),    # Cyan
    49: (128, 128, 128)   # Gray
}

# Sample 2D NumPy array (replace with your data)
gray_image = mask_dcm.pixel_array[80].copy()

def map_mask_colors(gray_image):

    # Create an empty RGB image
    rgb_image = np.zeros((gray_image.shape[0], gray_image.shape[1], 3), dtype=np.uint8)

    # Apply the mapping
    for gray_value, rgb_color in gray_to_rgb.items():
        rgb_image[gray_image == gray_value] = rgb_color
    return rgb_image


In [ ]:
im=mask_dcm.pixel_array[80].copy()
im1 = np.where(im==1,1,0)
im2 = np.where(im==0,1,0)
im3 = np.where(im==17,17,0)
im4 = np.where(im==2,33,0)
show_n_images([map_mask_colors(im), map_mask_colors(im1),
               map_mask_colors(im3), map_mask_colors(im4),map_mask_colors(im2)])

In [ ]:
im=mask_dcm.pixel_array[160-62].copy()
im1 = np.where(im==1,1,0)
im2 = np.where(im==0,2,0)
im3 = np.where(im==17,17,0)
im4 = np.where(im==2,33,0)
show_n_images([map_mask_colors(im), map_mask_colors(im1),
               map_mask_colors(im3), map_mask_colors(im4),map_mask_colors(im2)])

In [ ]:
np.unique(mask_dcm.pixel_array.flatten())

## Organizing DCE-MRI Slices of all acquisitions by Z-Axis Position

We observe that the slices are ordered along the **Z-axis**, ranging from `-78.41953` to `80.58047`. Using this information, we can correctly **organize all 1120 DCE-MRI slices** into their respective positions.

### **Steps for Organizing Slices**
1. **Initialize Storage Arrays**  
   - We create **7 zero arrays** (one for each acquisition), each with a shape of `(160, 384, 384)`, corresponding to **160 slices per acquisition**.

2. **Assign Slices to Their Correct Positions**  
   - We iterate through all slices and **insert each slice** into its appropriate position based on its **Z-location**.

This structured approach ensures that the DCE-MRI dataset is accurately **reconstructed in spatial order**.
To keep coherence, we sort all mri slices of mask and acuisition in an increasing order.